In [1]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods 
import make_question as question_maker
import vectorization
import sequence_handler
import corpus_handling_methods
import pandas as pd
import ranking
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
TFIDF_vectorizer = TfidfVectorizer()

import numpy as np
from bert_serving.client import BertClient
bc = BertClient()

In [2]:
dataset = pd.read_csv("../../../Helpers/Title_link_merged_corpus.csv")
dataset = dataset.iloc[:,2:]
dataset = dataset.rename(columns={"Article": "Data"})
dataset.Data = dataset.Data.apply(lambda x: corpus_handling_methods.clean_text(x))
dataset.head()

,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [3]:
split_corpus = corpus_handling_methods.corpus_split(dataset, 8)
split_corpus.head()

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,設定 順序 MobiControl 設定 順序 行い 端末 OS,0
2,対象 オンプレミス MobiControl サーバ のみ 必要 な 設定,0
3,事項 iOS 端末 macOS コンピュータ 必須 接続 プロファイル,0
4,作成 ADDS Active Directory Domain Service サーバ 間,0


In [4]:
perpage_dataset = corpus_handling_methods.corpus_per_page(split_corpus)
perpage_dataset.head() 

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,4


In [5]:
sample_size = 10
questions_samples = question_maker.question_dataframe_generator_1000(split_corpus, sample_size)
questions_samples.head()

,Question,PageID
0,など メモ 当該 端末 備忘録 メモ 残し 何ができますか,247
1,MobiControl Manual WordPress 構築 アラートイベント 発生 たら...,41
2,から 入力 なけれ ば 有効 期間 切れ 関する情報を教えてください,433
3,有効 期間 少なく なっ たら 再 発行 なにで,13
4,ケーブル Bluetooth 接続 ファイル 取り出し ない ウィルス 使うには,238


In [6]:
vec = vectorization.vector_fit(TFIDF_vectorizer, split_corpus)

In [30]:
sample_count = 0
sum_score = 0
container = []

for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    #ranks = ranking.crude_ranks(perpage_sequence_match, query, vec)
    #filtered_ranks = ranking.filtering_ranks(ranks, perpage_sequence_match, query, vec)
    
    ranks = bert_vector_to_rank(perpage_sequence_match)
    
    page_answers = []
    prediction_scores = []
    for ids, score in ranks:
        page_answers.append(ids)
        prediction_scores.append(score)
    MRR_score = ranking.mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score += MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
    sample_count += 1
    
result = pd.DataFrame(container, columns=['score', 'actual_answer',
'page_answers', 'prediction_scores', 'query'])
result.to_csv('seq_matcher_TFIDF_Vectorizer_performance.csv')
score = sum_score/sample_count
score


など メモ 当該 端末 備忘録 メモ 残し 何ができますか 247


/home/iftekhar/myworkplace/environments/experiment_env/lib/python3.6/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


MobiControl Manual WordPress 構築 アラートイベント 発生 たら どうでしたか 41
から 入力 なけれ ば 有効 期間 切れ 関する情報を教えてください 433
有効 期間 少なく なっ たら 再 発行 なにで 13
ケーブル Bluetooth 接続 ファイル 取り出し ない ウィルス 使うには 238
遷移 画面 れ デプロイメントサーバー 項 デプロイメントサーバ 名前 使用しますか 425
by Penetrate Limits Co Tokyo MobiControl v どうでしたか 124
有効 期間 残り 日数 指定 LDAP 証明 使いたいです 13
Catalog authentication 補足 iOS 端末 本人 認証 関する情報を教えてください 43
Shift 項目 説明 スティッキーキー 有効 キー 有効 何ですか 315
実行 複数 スクリプト 文 入力 可能 スクリプト どうですか 400
jp com アクセス ログイン 画面 な ログイン 何ですか 230


0.19416666666666668

In [7]:
sample_count = 0
sum_score = 0
container = []

for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    break

など メモ 当該 端末 備忘録 メモ 残し 何ができますか 247


In [ ]:
perpage_sequence_match

In [29]:
def bert_vector_to_rank(perpage_sequence_match, topk = 3):
    sentence_list = []
    ids_list = []
    rank = []
    for ids, items in perpage_sequence_match:
        for sentences in items:
            sentence_list.append(sentences)
            ids_list.append(ids)
    selected_vectors = bc.encode(sentence_list) 
    score = np.sum(X * selected_vectors, axis=1) / np.linalg.norm(selected_vectors, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
    for idx in topk_idx:
        rank.append([ids_list[idx], score[idx]])
    return rank